**WELCOME TO PART II HACKERS**

> We will quickly introduce you to the various packages you will need to help you develop the Geographical Information System application/website/dashboard using Colab


---



1.   GEEMAP is a Python package for interactive geospatial analysis and visualization with Google Earth Engine (GEE), which is a cloud computing platform with a multi-petabyte catalog of satellite imagery and geospatial datasets. You will need to install it to interact with readily available satellite image data.




In [1]:
%pip install geemap

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install PyCRS

  Using cached PyCRS-1.0.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
import geemap

In [4]:
import ee

In [5]:
import os

In [6]:
import geopandas

In [7]:
from geemap import geojson_to_ee, ee_to_geojson


2.   Authentication with Earth Engine is required to connect your project with Earth Engine



Run the ee.Authenticate function to authenticate your access to Earth Engine servers and ee.Initialize to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. More at: https://developers.google.com/earth-engine/guides/python_install

In case ee.Initialize() is not running, you need to create your own cloud EE project and enable API direclty to allow Colab to use EE features through this link: https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup (follow the link on the section "Create a Cloud project" then proceed with the next section "Enable the Earth Engine API")

In [8]:
ee.Authenticate()

True

In [9]:
ee.Initialize(project="digital-yeti-417904")

Alternatively: Open GEE Code Editor https://code.earthengine.google.com/, set up your own project manually (set up a non commerciable project) and enable permission for the GEE APIs for Colab.


```
ee.Initialize (project='your project name')
```





---



Now let's start with basic vizualization process, first you will be asked to upload the batch of shapefiles provided to you in your local file folder in collab. The Brazil.shp file serves as the official country boundary of our area of interest.

In [10]:
brazil_shapefile = geemap.shp_to_ee('Brazil.shp')

Next, let's see how the land cover was in January 1st 2004 in Brazil using the MODIS MCD12Q1 Type 1 Land Cover

In [11]:
Map = geemap.Map()

landcover = ee.Image('MODIS/006/MCD12Q1/2004_01_01').select('LC_Type1')

igbpLandCoverVis = {
    'min': 1.0,
    'max': 17.0,
    'palette': [
        '05450a',
        '086a10',
        '54a708',
        '78d203',
        '009900',
        'c6b044',
        'dcd159',
        'dade48',
        'fbff13',
        'b6ff05',
        '27ff87',
        'c24f44',
        'a5a5a5',
        'ff6d4c',
        '69fff8',
        'f9ffa4',
        '1c0dff',
    ],
}
brazil_lc = landcover.clip(brazil_shapefile)
Map.setCenter(-55, -10, 4)
Map.addLayer(brazil_lc, igbpLandCoverVis, 'MODIS Land Cover')

Map

Map(center=[-10, -55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [12]:
Map.add_legend(builtin_legend='MODIS/051/MCD12Q1')

Next, we can compute basic statistics of how much hectares of each LC type was in Brazil in 2004. This line will allow you to donwload the results in your colab directory

In [13]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
global_stats = os.path.join(out_dir, 'global_stats_sum.csv')

# statistics_type can be either 'SUM' or 'PERCENTAGE'
# denominator can be used to convert square meters to other areal units, such as square kilimeters
geemap.zonal_statistics_by_group(
    landcover,
    brazil_shapefile,
    global_stats,
    statistics_type='SUM',
    denominator=1000000,
    decimal_places=2,
)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /home/tlrmendonca/Downloads/global_stats_sum.csv


Better: you can just count the number of pixel for each class. As a pixel for a MODIS image is 250 m x 250 m, the area for one class will be number of pixel x 250 x 250

In [18]:
# Calculate pixel areas
areaImage = ee.Image.pixelArea().addBands(lc14)

# Reduce the region to get summed areas
areas = areaImage.reduceRegion(
    reducer=ee.Reducer.sum().group(groupField=1, groupName='class'),
    geometry=fc.geometry(),
    scale=500,
    maxPixels=1e10
)

# Print the result
print(areas.getInfo())

{'groups': [{'class': 1, 'sum': 309980291.4876225}, {'class': 2, 'sum': 3533992334757.3857}, {'class': 4, 'sum': 46283395108.926895}, {'class': 5, 'sum': 5124016513.336519}, {'class': 6, 'sum': 43589139744.15362}, {'class': 7, 'sum': 3595515855.2227325}, {'class': 8, 'sum': 389069629629.309}, {'class': 9, 'sum': 2089060513388.6626}, {'class': 10, 'sum': 1807128008565.1658}, {'class': 11, 'sum': 59962619903.20342}, {'class': 12, 'sum': 286494945558.9042}, {'class': 13, 'sum': 39722254017.48277}, {'class': 14, 'sum': 83534346409.29033}, {'class': 16, 'sum': 3117226010.5401964}, {'class': 17, 'sum': 79723507550.9369}]}


Now, let's pull a time series vizualization of the landcover in Brazil



1.   This is the long code to help you understand the process



In [17]:
Map = geemap.Map()

lc03 = ee.Image('MODIS/006/MCD12Q1/2003_01_01').select('LC_Type1')
lc14 = ee.Image('MODIS/006/MCD12Q1/2014_01_01').select('LC_Type1')
lc15 = ee.Image('MODIS/006/MCD12Q1/2015_01_01').select('LC_Type1')
lc16 = ee.Image('MODIS/006/MCD12Q1/2016_01_01').select('LC_Type1')
lc17 = ee.Image('MODIS/006/MCD12Q1/2017_01_01').select('LC_Type1')
lc18 = ee.Image('MODIS/006/MCD12Q1/2018_01_01').select('LC_Type1')
lc19 = ee.Image('MODIS/006/MCD12Q1/2019_01_01').select('LC_Type1')
lc20 = ee.Image('MODIS/006/MCD12Q1/2020_01_01').select('LC_Type1')

igbpLandCoverVis = {
    'min': 1.0,
    'max': 17.0,
    'palette': [
        '05450a',
        '086a10',
        '54a708',
        '78d203',
        '009900',
        'c6b044',
        'dcd159',
        'dade48',
        'fbff13',
        'b6ff05',
        '27ff87',
        'c24f44',
        'a5a5a5',
        'ff6d4c',
        '69fff8',
        'f9ffa4',
        '1c0dff',
    ],
}
brazil_lc03 = lc03.clip(brazil_shapefile)
brazil_lc14 = lc14.clip(brazil_shapefile)
brazil_lc15 = lc15.clip(brazil_shapefile)
brazil_lc16 = lc16.clip(brazil_shapefile)
brazil_lc17 = lc17.clip(brazil_shapefile)
brazil_lc18 = lc18.clip(brazil_shapefile)
brazil_lc19 = lc19.clip(brazil_shapefile)
brazil_lc20 = lc20.clip(brazil_shapefile)

Map.setCenter(-55, -10, 4)
Map.addLayer(brazil_lc14, igbpLandCoverVis, 'MODIS Land Cover 2014')
Map.addLayer(brazil_lc15, igbpLandCoverVis, 'MODIS Land Cover 2015')
Map.addLayer(brazil_lc16, igbpLandCoverVis, 'MODIS Land Cover 2016')
Map.addLayer(brazil_lc17, igbpLandCoverVis, 'MODIS Land Cover 2017')
Map.addLayer(brazil_lc18, igbpLandCoverVis, 'MODIS Land Cover 2018')
Map.addLayer(brazil_lc19, igbpLandCoverVis, 'MODIS Land Cover 2019')
Map.addLayer(brazil_lc20, igbpLandCoverVis, 'MODIS Land Cover 2020')

Map.addLayerControl()

Map

Map(center=[-10, -55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

NEXT is how you import and add population density to the previous map

In [19]:
dataset = ee.ImageCollection('WorldPop/GP/100m/pop')

# Visualization parameters
visualization = {
  'bands': ['population'],
  'min': 0.0,
  'max': 50.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
}

# Set center of the map
center = [34.769, 113.643]
zoom = 7

# Add the layer to the map
image = dataset.mean()
Map.addLayer(image, visualization, 'Population')




2.   This is how it looks like when it is more intricate (you might laugh cause it is not at all intricate for you)


In [20]:
# Load MODIS land cover images for multiple years
years = range(2002, 2021)  # 2002 to 2020
lc_images = []

for year in years:
    lc_image = ee.Image(f"MODIS/006/MCD12Q1/{year}_01_01").select('LC_Type1')
    lc_images.append(lc_image)

# Visualization parameters
igbpLandCoverVis = {
    'min': 1.0,
    'max': 17.0,
    'palette': [
        '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044',
        'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
        'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff'
    ]
}

# Clip the land cover images to Brazil
brazil_lc_images = [lc.clip(brazil_shapefile) for lc in lc_images]

# Display the land cover images for each year
Map = geemap.Map(center=[-10, -55], zoom=4)

for i, lc_image in enumerate(brazil_lc_images):
    Map.addLayer(lc_image, igbpLandCoverVis, f'MODIS Land Cover {years[i]}')

legend_keys = ['Evergreen Needleleaf Forests', 'Evergreen Broadleaf Forests', 'Deciduous Needleleaf Forests', 'Deciduous Broadleaf Forests',
               'Mixed Forests', 'Closed Shrublands', 'Open Shrublands', 'Woody Savannas',
               'Savannas', 'Grasslands', 'Permanent Wetlands', 'Croplands', 'Urban and Built-up',
               'Cropland/Natural Vegetation Mosaics', 'Permanent Snow and Ice', 'Barren', 'Water Bodies']
igbpLandCoverVisu = ['05450a', '086a10', '54a708', '78d203', '009900', 'c6b044',
        'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
        'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff']

Map.add_legend(keys=legend_keys, colors=igbpLandCoverVisu, position='bottomright')
Map.addLayerControl()

Map


Map(center=[-10, -55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

You can also suggest a side by side vizualization of the land cover, allowing one to vizaully assess the extent of degradation over time.

In [21]:
Map = geemap.Map(center=[-9, -51], zoom=10)

left_layer = geemap.ee_tile_layer(brazil_lc03, igbpLandCoverVis, "MODIS")
right_layer = geemap.ee_tile_layer(brazil_lc20, igbpLandCoverVis, "MODIS")
Map.split_map(left_layer, right_layer)
Map.add_legend(
    title="Land cover 2003", builtin_legend='MODIS/006/MCD12Q1', position='bottomleft'
)
Map.add_legend(
    title="Land cover 2020",
    builtin_legend='MODIS/006/MCD12Q1',
    position='bottomright',
)


Map

Map(center=[-9, -51], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

This part is more fun!!! Using the timeseries tool to compare 20 years of landcover against each other.

In [15]:
col = ee.ImageCollection('MODIS/006/MCD12Q1').select('LC_Type1');

In [16]:
# A FeatureCollection defining Brazil boundary.
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    'country_na == "Brazil"'
)

# Clip the DEM by the Southeast Asia boundary FeatureCollection.
# Iterate over the ImageCollection and clip each image to the FeatureCollection.
col_clip = col.map(lambda img: img.clipToCollection(fc))
DOY = col_clip.filterDate('2002-01-01', '2021-01-01');

In [22]:
distinctDOY = col.filterDate('2002-01-01', '2021-01-01');

Check the number of image you have between that range:

In [23]:
num_images = distinctDOY.size().getInfo()

print("Number of images in the collection:", num_images)

Number of images in the collection: 19


In [25]:
layer_names = ['MODIS ' + str(year) for year in range(2002, 2021)]
print(layer_names)

['MODIS 2002', 'MODIS 2003', 'MODIS 2004', 'MODIS 2005', 'MODIS 2006', 'MODIS 2007', 'MODIS 2008', 'MODIS 2009', 'MODIS 2010', 'MODIS 2011', 'MODIS 2012', 'MODIS 2013', 'MODIS 2014', 'MODIS 2015', 'MODIS 2016', 'MODIS 2017', 'MODIS 2018', 'MODIS 2019', 'MODIS 2020']


In [55]:
Map = geemap.Map(center=[-9, -51], zoom=10)
Map.add_basemap('HYBRID')
Map.centerObject(brazil_shapefile)

images = geemap.modis_timeseries(asset_id='MODIS/006/MCD12Q1', band_name='LC_Type1', roi=brazil_shapefile,
                                 start_year = 2003, end_year = 2021,
                                 start_date= '01-01', end_date= '12-31')
Map.ts_inspector(left_ts=DOY, left_names=layer_names, left_vis=igbpLandCoverVis, left_index=0,
                 right_ts=DOY, right_names=layer_names, right_vis=igbpLandCoverVis, right_index=-1,
                 width='130px', date_format='YYYY', add_close_button=False)
Map.add_legend(title="MODIS Land Cover", builtin_legend='MODIS/006/MCD12Q1')
Map.addLayerControl

Map

Map(center=[-9, -51], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=L…

One way to calculate area of a specific Image in Brazil

THIS IS THE CODE TO VIZUALIZE THE BURNED AREAS

In [27]:
# Define the dataset and filter by date
dataset = ee.ImageCollection('MODIS/061/MCD64A1').filter(ee.Filter.date('2017-01-01', '2018-05-01'))
burnedArea = dataset.select('BurnDate')

# A FeatureCollection defining Brazil boundary.
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    'country_na == "Brazil"'
)

# Clip the burned Area by the Brazil boundary FeatureCollection.
# Iterate over the ImageCollection and clip each image to the FeatureCollection.
ba_clip = burnedArea.map(lambda img: img.clipToCollection(fc))


# Visualization parameters
burnedAreaVis = {
    'min': 30.0,
    'max': 341.0,
    'palette': ['4e0400', '951003', 'c61503', 'ff1901']
}

# Create a map
Map = geemap.Map(center=[-10, -55], zoom=4)

# Add burned area layer to the map
Map.addLayer(ba_clip, burnedAreaVis, 'Burned Area')
Map.addLayer(brazil_shapefile, name='Brazil',opacity=0.5)
Map.addLayerControl()

# Display the map
Map

Map(center=[-10, -55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Lets talk about metheorogical data

### Temperature

In [28]:
Map = geemap.Map()

# temprature = ee.Image('NOAA/CFSV2/FOR6H').select('Temperature_height_above_ground')
temperature = ee.ImageCollection("NOAA/CFSV2/FOR6H").filter(ee.Filter.date('2023-01-01', '2024-01-01')).select('Temperature_height_above_ground')
temp_image = temperature.mean()

tempratureVis = {
    'min': 189.8,
    'max': 334.89,
    'palette': ['05450a','086a10','54a708','78d203','009900','c6b044','dcd159','dade48','fbff13','b6ff05','27ff87','c24f44','a5a5a5','ff6d4c','69fff8','f9ffa4','1c0dff',],
}

brazil_temp = temp_image.clip(brazil_shapefile)
Map.setCenter(-55, -10, 4)
Map.addLayer(brazil_temp, tempratureVis, 'Temperature 2m Above Ground')

Map

Map(center=[-10, -55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

### Wind velocity

In [29]:
Map = geemap.Map()

# temprature = ee.Image('NOAA/CFSV2/FOR6H').select('Temperature_height_above_ground')
v_wind_speed = ee.ImageCollection("NOAA/CFSV2/FOR6H").filter(ee.Filter.date('2023-01-01', '2024-01-01')).select('v-component_of_wind_height_above_ground')
u_wind_speed = ee.ImageCollection("NOAA/CFSV2/FOR6H").filter(ee.Filter.date('2023-01-01', '2024-01-01')).select('u-component_of_wind_height_above_ground')
wind_image = (v_wind_speed.max().pow(2)).add(u_wind_speed.max().pow(2))
wind_speed_image = wind_image.sqrt()

windSpeedVis = {
    'min': -10.8,
    'max': 10.89,
    'palette': ['yellow', 'green'],
}

brazil_wind = wind_speed_image.clip(brazil_shapefile)
Map.setCenter(-55, -10, 4)
Map.addLayer(brazil_wind, windSpeedVis, 'Wind Velocity 10m Above Ground')

Map

Map(center=[-10, -55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [54]:
Map = geemap.Map()

dataset = ee.ImageCollection("projects/climate-engine-pro/assets/ce-merra2_fwi-daily")
image = dataset.filterDate('2022-12-01', '2023-02-01').select('FWI').max()

vis_params = {
    'min': 0,
    'max': 50,
    'palette': ['green', 'yellow', 'orange', 'red'],
}

image = image.clip(brazil_shapefile)
Map.setCenter(-55, -10, 4)
Map.addLayer(image, vis_params, 'FWI')

# legend
legend_keys = ['Low', 'Medium', 'High']
gradient_colors = [(0, 255, 0), (255, 255, 0), (255, 0, 0)]  # RGB for green, yellow, red

# Define the title for the legend
legend_title = 'FWI'

# Add the gradient legend to the map
Map.add_legend(title=legend_title, keys=legend_keys, colors=gradient_colors, position='bottomright')

Map

Map(center=[-10, -55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [34]:
dataset.first()